In [1]:
import warnings, os
import datetime as dt
import itertools as it
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [6]:
import MetaTrader5 as mt5

# Connection

In [ ]:
RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

PATH_3 = r'C:\Program Files\MetaTrader 5 _3\terminal64.exe'

assert mt5.initialize(PATH_3, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H3','D1','W1']

# Consume

In [9]:
def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
    Pipe = []
    for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
        timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
        Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
        Raw.insert(0, 'a',          Raw.index + 1)
        Raw.insert(1, 'z',          Raw.index - Raw.index.max())
        Raw.insert(2, 'symbol',     symbol)
        Raw.insert(3, 'tf',         TF)
        Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
        Pipe.append(Raw)
    pass
    return pd.concat(Pipe, ignore_index=1)

Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
Raw.head()

,a,z,symbol,tf,datetime,time,open,high,low,close,tick_volume,spread,real_volume
0,1,-99998,WIN$,M5,2021-11-25 12:30:00,1637843400,153184.0,153235.0,153113.0,153199.0,30898,1,108074
1,2,-99997,WIN$,M5,2021-11-25 12:35:00,1637843700,153206.0,153465.0,153163.0,153163.0,50380,1,186621
2,3,-99996,WIN$,M5,2021-11-25 12:40:00,1637844000,153163.0,153307.0,153055.0,153235.0,40566,1,137131
3,4,-99995,WIN$,M5,2021-11-25 12:45:00,1637844300,153242.0,153580.0,153170.0,153522.0,56134,1,202212
4,5,-99994,WIN$,M5,2021-11-25 12:50:00,1637844600,153522.0,153551.0,153235.0,153242.0,37612,1,126769


In [ ]:
raise Exception('STOP')

# Save

In [ ]:
# Raw.to_csv('Raw.csv', index=False)

# Load

In [ ]:
Raw = pd.read_csv('Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=[*SYMBOLS], TFRAMES=[*TFRAMES])
Query.head()

# Check

In [33]:
Raw.groupby('tf', sort=0)[['datetime']].count()

,datetime
tf,
M5,99999
M15,46109
H1,11866
H3,4408
D1,1246
W1,262


In [34]:
Raw.groupby('tf', sort=0)[['datetime']].first()

,datetime
tf,
M5,2021-11-25 12:30:00
M15,2020-07-10 09:00:00
H1,2020-07-10 09:00:00
H3,2020-07-10 09:00:00
D1,2020-07-10 00:00:00
W1,2020-07-05 00:00:00
